# Predicción de palabras con redes neuronales

Este proyecto se enfoca en la adquisición de un conjunton de datos para posteriormente introducir los embeddings en una red neuronal, permitiendo predecir la siguiente palabra basándose en dichas entradas. Para este caso se hace uso de un corpus de canciones (se tomarán únicamente canciones en inglés) de un total de 79 generos musicales. Ese corpus contiene 379,893 letras de canciones de 4,239 artistas.


Adquisición de los datos.

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("neisse/scrapped-lyrics-from-6-genres")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'scrapped-lyrics-from-6-genres' dataset.
Path to dataset files: /kaggle/input/scrapped-lyrics-from-6-genres


In [2]:
import pandas as pd

file_name = 'lyrics-data.csv'
complete_route = path + '/' + file_name
df = pd.read_csv(complete_route)
df.head()

,ALink,SName,SLink,Lyric,language
0,/ivete-sangalo/,Arerê,/ivete-sangalo/arere.html,"Tudo o que eu quero nessa vida,\nToda vida, é\...",pt
1,/ivete-sangalo/,Se Eu Não Te Amasse Tanto Assim,/ivete-sangalo/se-eu-nao-te-amasse-tanto-assim...,Meu coração\nSem direção\nVoando só por voar\n...,pt
2,/ivete-sangalo/,Céu da Boca,/ivete-sangalo/chupa-toda.html,É de babaixá!\nÉ de balacubaca!\nÉ de babaixá!...,pt
3,/ivete-sangalo/,Quando A Chuva Passar,/ivete-sangalo/quando-a-chuva-passar.html,Quando a chuva passar\n\nPra quê falar\nSe voc...,pt
4,/ivete-sangalo/,Sorte Grande,/ivete-sangalo/sorte-grande.html,A minha sorte grande foi você cair do céu\nMin...,pt


Verificar los lenguajes disponibles en el dataset

In [3]:
languages = df['language'].unique()
print("Lenguajes disponibles: ",languages)

Lenguajes disponibles:  ['pt' 'es' 'en' nan 'it' 'gl' 'fr' 'de' 'tl' 'et' 'fi' 'pl' 'da' 'st' 'sv'
 'ro' 'af' 'no' 'eu' 'rw' 'sw' 'ga' 'cy' 'ca' 'ny' 'ko' 'ar' 'gd' 'tr'
 'id' 'su' 'lg' 'ru' 'nl' 'sq' 'is' 'cs' 'jw' 'lv' 'hu' 'ms' 'ku' 'zh'
 'hr' 'ht' 'fa' 'mg' 'vi' 'ja' 'hmn' 'sr' 'iw' 'sl']


En este proyecto únicamente se realizarán predicciones en inglés. Debido a las restricciones de memoria, se limita únicamente a 2000 canciones. 

In [4]:
df_en = df[df['language'] == 'en'].head(2000)
print("Longitud del dataset: ",len(df_en))
df_en.head()

Longitud del dataset:  2000


,ALink,SName,SLink,Lyric,language
69,/ivete-sangalo/,Careless Whisper,/ivete-sangalo/careless-whisper.html,I feel so unsure\nAs I take your hand and lead...,en
86,/ivete-sangalo/,Could You Be Loved / Citação Musical do Rap: S...,/ivete-sangalo/could-you-be-loved-citacao-musi...,"Don't let them fool, ya\nOr even try to school...",en
88,/ivete-sangalo/,Cruisin' (Part. Saulo),/ivete-sangalo/cruisin-part-saulo.html,"Baby, let's cruise, away from here\nDon't be c...",en
111,/ivete-sangalo/,Easy,/ivete-sangalo/easy.html,"Know it sounds funny\nBut, I just can't stand ...",en
140,/ivete-sangalo/,For Your Babies (The Voice cover),/ivete-sangalo/for-your-babies-the-voice-cover...,You've got that look again\nThe one I hoped I ...,en


Limpieza de la columna 'Lyric' 

In [5]:
import re
from nltk.tokenize import word_tokenize
import numpy as np
import nltk

nltk.download('punkt_tab')

def preprocessing(text):
    if pd.isna(text):
        return []

    text = str(text).lower()
    text = re.sub(r'[^a-zA-Z\s\']', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()

    tokens = word_tokenize(text)
    return tokens

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [6]:
df_en['tokens'] = df_en['Lyric'].apply(preprocessing)
df_en['tokens'].head()

,tokens
69,"[i, feel, so, unsure, as, i, take, your, hand,..."
86,"[do, n't, let, them, fool, ya, or, even, try, ..."
88,"[baby, let, 's, cruise, away, from, here, do, ..."
111,"[know, it, sounds, funny, but, i, just, ca, n'..."
140,"[you, 've, got, that, look, again, the, one, i..."


Creación de la ventana de contexto

In [7]:
WINDOW_SIZE = 5

In [8]:
# Generar la secuencia de palabras y su predicción
def create_windows(tokens, window_size=WINDOW_SIZE):
    X = []
    y = []
    for i in range(len(tokens) - window_size):
        window = tokens[i : i + window_size]       # secuencia de contexto
        next_word = tokens[i + window_size]        # palabra objetivo
        X.append(window)
        y.append(next_word)
    return X, y


sample_tokens = df_en['tokens'].iloc[38] #Ejemplo de la fila 38
X, y = create_windows(sample_tokens, WINDOW_SIZE)

print("Ejemplo de ventana:", X[0])
print("Palabra a predecir:", y[0])


Ejemplo de ventana: ['remember', 'those', 'walls', 'i', 'built']
Palabra a predecir: well


Aplicación a todo el dataset

In [9]:
all_X = []
all_y = []

for tokens in df_en['tokens']:
    X, y = create_windows(tokens, WINDOW_SIZE)
    all_X.extend(X)
    all_y.extend(y)

print("Total de ejemplos creados:", len(all_X))
print("Ejemplo:", all_X[100], "->", all_y[100])


Total de ejemplos creados: 840130
Ejemplo: ['of', 'a', 'good', 'friend', 'to'] -> the


Procesamiento de embeddings

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


tokenizer = Tokenizer()

In [ ]:
tokenizer.fit_on_texts(all_X + all_y)

X_seq = tokenizer.texts_to_sequences(all_X)
y_seq = tokenizer.texts_to_sequences(all_y)

# Aplanar las secuencias
y_seq = np.array([seq[0] for seq in y_seq if len(seq) > 0])
X_seq = X_seq[:len(y_seq)]

# Asegurar que las secuencias tengan el mismo tamaño
X_seq = pad_sequences(X_seq, maxlen=WINDOW_SIZE, padding="pre")

print("X shape:", X_seq.shape)
print("y shape:", y_seq.shape)




X shape: (840125, 5)
y shape: (840125,)


Creación del modelo de red neuronal

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense

VOCAB_SIZE = len(tokenizer.word_index) + 1  # número total de palabras

model = Sequential([
    Embedding(input_dim=VOCAB_SIZE, output_dim=100, input_length=WINDOW_SIZE),
    Flatten(),
    Dense(256, activation="relu"),
    Dense(VOCAB_SIZE, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Entrenamiento del modelo

Para este caso se eligieron 5 épocas, con un tamaño de lote de 256 y para la validación se usó un 20% de los datos. 

In [11]:
history = model.fit(X_seq, y_seq, epochs=5, batch_size=256, validation_split=0.2)

Epoch 1/5
2626/2626 ━━━━━━━━━━━━━━━━━━━━ 777s 295ms/step - accuracy: 0.0619 - loss: 6.4539 - val_accuracy: 0.0442 - val_loss: 6.5919
Epoch 2/5
2626/2626 ━━━━━━━━━━━━━━━━━━━━ 799s 294ms/step - accuracy: 0.1236 - loss: 5.5278 - val_accuracy: 0.0451 - val_loss: 6.8211
Epoch 3/5
2626/2626 ━━━━━━━━━━━━━━━━━━━━ 825s 303ms/step - accuracy: 0.1753 - loss: 4.9644 - val_accuracy: 0.0419 - val_loss: 7.1470
Epoch 4/5
2626/2626 ━━━━━━━━━━━━━━━━━━━━ 792s 299ms/step - accuracy: 0.2275 - loss: 4.4653 - val_accuracy: 0.0395 - val_loss: 7.5839
Epoch 5/5
2626/2626 ━━━━━━━━━━━━━━━━━━━━ 791s 295ms/step - accuracy: 0.2779 - loss: 4.0306 - val_accuracy: 0.0366 - val_loss: 8.0689


Gaurdado del modelo entrenado

Para evitar nuevamente reentrenar la red, se guarda el modelo junto con el tokenizador. 

In [14]:
import pickle
# Guardar el modelo entrenado
model.save("modelo_lyrics.h5")
# Guardar tokenizer
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

En dado caso de que solamente se quiera hacer uso del modelo se puede ejecutar la siguiente celda de código.

In [15]:
# Cargar el modelo entrenado
from tensorflow.keras.models import load_model

model = load_model("modelo_lyrics.h5")
# Cargar tokenizer
with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

Función para predecir una frase

In [16]:
def predict_next_word(model, tokenizer, text, window_size=5):
    tokens = preprocessing(text)

    if len(tokens) < window_size:
        # si la frase es más corta se rellena
        tokens = ["<pad>"] * (window_size - len(tokens)) + tokens

    # Últimas palabras de la ventana
    window = tokens[-window_size:]

    seq = tokenizer.texts_to_sequences([window])
    seq = pad_sequences(seq, maxlen=window_size, padding="pre")

    pred = model.predict(seq, verbose=0)
    next_word_id = np.argmax(pred, axis=-1)[0]

    # Se convierte el índice en una palabra
    for word, idx in tokenizer.word_index.items():
        if idx == next_word_id:
            return word
    return None

Prueba de predicción

In [17]:
STRING_LEN = 8
text_input = "I"
for i in range(STRING_LEN):
  predicted_word = predict_next_word(model, tokenizer, text_input, WINDOW_SIZE)
  text_input += ' ' + predicted_word
  print("Texto:", text_input)
  print("Siguiente palabra predicha:", predicted_word)

Texto: I abusin
Siguiente palabra predicha: abusin
Texto: I abusin homeboy
Siguiente palabra predicha: homeboy
Texto: I abusin homeboy border
Siguiente palabra predicha: border
Texto: I abusin homeboy border swizzie
Siguiente palabra predicha: swizzie
Texto: I abusin homeboy border swizzie royce
Siguiente palabra predicha: royce
Texto: I abusin homeboy border swizzie royce tower
Siguiente palabra predicha: tower
Texto: I abusin homeboy border swizzie royce tower ohhhhhhhhhhhhhhh
Siguiente palabra predicha: ohhhhhhhhhhhhhhh
Texto: I abusin homeboy border swizzie royce tower ohhhhhhhhhhhhhhh climbin
Siguiente palabra predicha: climbin


Evaluación con perplexity en train/test

El conjunto de datos siguie siendo muy grande, por lo que hace uso de mucha memoria RAM. 
Para evitar el uso excesivo de memoria, se calcula la perplejidad por lotes. 

In [19]:
import numpy as np

def compute_perplexity_batch(model, X, y, batch_size=512):
    N = len(y)
    total_log_prob = 0.0

    for i in range(0, N, batch_size):
        X_batch = X[i:i+batch_size]
        y_batch = y[i:i+batch_size]
        preds = model.predict(X_batch, batch_size=batch_size, verbose=0)

        probs = preds[np.arange(len(y_batch)), y_batch]

        probs = np.where(probs > 0, probs, 1e-10)

        total_log_prob += np.log(probs).sum()

    ppl = np.exp(- total_log_prob / N)
    return ppl


In [20]:
split = int(len(X_seq) * 0.8)
X_train, X_val = X_seq[:split], X_seq[split:]
y_train, y_val = y_seq[:split], y_seq[split:]

train_ppl = compute_perplexity_batch(model, X_train, y_train, batch_size=512)
val_ppl = compute_perplexity_batch(model, X_val, y_val, batch_size=512)

print(f"Train PPL: {train_ppl:.2f}")
print(f"Validation PPL: {val_ppl:.2f}")


Train PPL: 18461.53
Validation PPL: 18461.87


## Conclusión 

Los resultados obtenidos en el cálculo de la perplejidad son considerablemente altos, lo que indica que el modelo está realizando predicciones poco precisas, esencialmente "adivinando" la siguiente palabra. Este comportamiento puede atribuirse a diversos factores, entre ellos la simplicidad de la arquitectura utilizada: Embedding → Flatten → Dense. 

Para mejorar el rendimiento, sería recomendable sustituir la capa densa por una capa recurrente, como LSTM o GRU.
Además, el número de épocas de entrenamiento fue limitado, lo cual influye en la capacidad del modelo para aprender patrones complejos. Un mayor número de épocas podría mejorar los resultados, aunque implicaría un incremento en el tiempo de cómputo y en el uso de recursos.


A pesar de sus limitaciones, esta práctica matiene las bases de los principios fundamentales de la predicción de texto, y permite realizar comparaciones con enfoques tradicionales como TF-IDF y la similitud por coseno.
